In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00


In [1]:
label_dict = {0: '(단언) 진술하기', 1: '(지시) 충고/제안하기', 2: '(단언) 주장하기', 3: '(지시) 질문하기',
              4: '(지시) 부탁하기', 5: '(단언) 반박하기', 6: '(표현) 감사하기', 7: '(표현) 사과하기',
              8: '(지시) 명령/요구하기', 9: '턴토크 사인(관습적 반응)',
              10: '(언약) 약속하기(제3자와)/(개인적 수준)', 11: '(언약) 거절하기', 12: '(표현) 인사하기'}

In [ ]:
from transformers import TextClassificationPipeline, BertTokenizerFast, TFBertForSequenceClassification
import os


# MODEL_NAME = 'fine-tuned-klue-bert-base'
# MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

MODEL_SAVE_PATH = '/home/inter/chat_MBTI/intent_classifier'

# Load Fine-tuning model
tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    return_all_scores=True
)

In [4]:
max_seq_len = 100
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print(label_dict[score])

In [8]:
while True :
  new_sentence = input('sentence > ')
  if new_sentence == '끝':
    break
  sentiment_predict(new_sentence)

sentence > 반대편 문을 사용해주세요


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 9s 9s/step
(지시) 부탁하기
sentence > 반대편 문을 사용하세요
1/1 [==============================] - 0s 440ms/step
(지시) 충고/제안하기
sentence > 반대편 문을 사용해주세요
1/1 [==============================] - 0s 420ms/step
(지시) 부탁하기
sentence > 끝


In [9]:
import numpy as np

def intent_predict(sentence):

  temp_li = []
  score_li = text_classifier(sentence)[0]
  for i in score_li:
    temp_li.append(i['score'])

  print(label_dict[int(score_li[temp_li.index(max(temp_li))]['label'][6:])])

In [10]:
intent_predict('하이')

(표현) 인사하기
